In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.messages import HumanMessage, AIMessage

from langgraph.checkpoint.memory import MemorySaver

from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent

from langsmith import utils

import os

In [3]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://eu.api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="langchain-agent-test"

os.environ["NVIDIA_API_KEY"] = "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"
os.environ["TAVILY_API_KEY"] = "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"

In [4]:
utils.tracing_is_enabled()

True

In [5]:
# using Tavily Search for online search to be used as an agent
search = TavilySearchResults(max_results=5)

search_results = search.invoke("What is the current weather in Barnet, London, United Kingdom")
print(search_results)

tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Barnet', 'region': 'Barnet, Greater London', 'country': 'United Kingdom', 'lat': 51.6445, 'lon': -0.2021, 'tz_id': 'Europe/London', 'localtime_epoch': 1732051879, 'localtime': '2024-11-19 21:31'}, 'current': {'last_updated_epoch': 1732051800, 'last_updated': '2024-11-19 21:30', 'temp_c': 1.2, 'temp_f': 34.2, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 11.0, 'wind_kph': 17.6, 'wind_degree': 334, 'wind_dir': 'NNW', 'pressure_mb': 1004.0, 'pressure_in': 29.65, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 93, 'cloud': 0, 'feelslike_c': -3.4, 'feelslike_f': 25.9, 'windchill_c': -2.8, 'windchill_f': 27.0, 'heatindex_c': 1.7, 'heatindex_f': 35.0, 'dewpoint_c': -1.6, 'dewpoint_f': 29.1, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 17.0, 'gust_kph': 27.4}}"}, {'url': 'https://www.ventusky.com/barnet', 'content': "Barnet 

Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://eu.api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('400 Client Error: Bad Request for url: https://eu.api.smith.langchain.com/runs/multipart', '{"detail":"Empty request"}')


In [6]:
# initialise NVIDIA model for chat
model = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")


In [8]:
response = model.invoke([HumanMessage(content = "Hi, my name is Deepak!")])
response.content

'Nice to meet you, Deepak! How are you doing today? Is there something I can help you with or would you like to just chat?'

In [9]:
# explicitly bind the tools to the model
model_with_tools = model.bind_tools(tools)

In [10]:
# tool does not get called with this query
response = model_with_tools.invoke([HumanMessage(content="I live in London, UK")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: That's a great city. What can I help you with today?
ToolCalls: []


In [11]:
# tool gets called with this query
response = model_with_tools.invoke([HumanMessage(content = "What is the time in Delhi, India right now?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'Delhi India time now'}, 'id': 'chatcmpl-tool-bfc3bbb7e42a4d528aee3a6736a253f9', 'type': 'tool_call'}]


In [15]:
# we are using Langraph to bind the tool with the model
# bind_tools get called internally
agent_executor = create_react_agent(model, tools)

In [13]:
# Search agent does not get invoked
response = agent_executor.invoke({"messages": [HumanMessage(content = "Hi, my name is Deepak!")]})

response["messages"]

[HumanMessage(content='Hi, my name is Deepak!', additional_kwargs={}, response_metadata={}, id='6e6dc2ff-617f-4e05-8d11-6bd51d1d7ec4'),
 AIMessage(content="Hello Deepak! It's nice to meet you. Is there something I can help you with today?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hello Deepak! It's nice to meet you. Is there something I can help you with today?", 'token_usage': {'prompt_tokens': 316, 'total_tokens': 337, 'completion_tokens': 21}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-9d53ffb6-876d-4264-b9d8-dca3fb65c068-0', usage_metadata={'input_tokens': 316, 'output_tokens': 21, 'total_tokens': 337}, role='assistant')]

In [14]:
# Search agent gets invoked
response = agent_executor.invoke({"messages":[HumanMessage(content="Which is the highest grossing movie of all time in the world?")]})

response["messages"]

[HumanMessage(content='Which is the highest grossing movie of all time in the world?', additional_kwargs={}, response_metadata={}, id='1072f7e4-1819-45bb-ac64-e822447423de'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-2f9e6228a3344a1d957c9653c108b5ae', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "highest grossing movie of all time"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-2f9e6228a3344a1d957c9653c108b5ae', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "highest grossing movie of all time"}'}}], 'token_usage': {'prompt_tokens': 322, 'total_tokens': 357, 'completion_tokens': 35}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-210ec42b-fc5d-4312-b5b8-a94d864b4882-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'highest grossing movie 

In [16]:
# streaming messages
for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="What is the temperature in Svalbard?")]}
):
    print(chunk)
    print("----")



{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-d8a0e5ff699443b483ba14fba1855be0', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Svalbard temperature"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-d8a0e5ff699443b483ba14fba1855be0', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "Svalbard temperature"}'}}], 'token_usage': {'prompt_tokens': 317, 'total_tokens': 350, 'completion_tokens': 33}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-7ad4dd99-3777-4a10-ac50-e51867b57544-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Svalbard temperature'}, 'id': 'chatcmpl-tool-d8a0e5ff699443b483ba14fba1855be0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 317, 'output_tokens': 33, 'total_tokens': 350}, role='assistant')]}}

In [18]:
# adding memory to the agent conversation
memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id":"abc123"}}

for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="Hi! this is Myra. I live in Edinburgh, Scotland")]}, config
):
    print(chunk)
    print("-----")

{'agent': {'messages': [AIMessage(content="Hello Myra! Nice to meet you! Edinburgh is a beautiful city, I've heard. How do you like living there? Is there something I can help you with or would you like to chat about something in particular?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hello Myra! Nice to meet you! Edinburgh is a beautiful city, I've heard. How do you like living there? Is there something I can help you with or would you like to chat about something in particular?", 'token_usage': {'prompt_tokens': 321, 'total_tokens': 366, 'completion_tokens': 45}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-25cb2e59-ec1e-4701-ab9d-91f4f6c77b46-0', usage_metadata={'input_tokens': 321, 'output_tokens': 45, 'total_tokens': 366}, role='assistant')]}}
-----


In [19]:
for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="What is the current temperature in my city?")]}, config
):
    print(chunk)
    print("-----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-bc78ce885e32428c9f4848249194279f', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "current temperature in Edinburgh, Scotland"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-bc78ce885e32428c9f4848249194279f', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "current temperature in Edinburgh, Scotland"}'}}], 'token_usage': {'prompt_tokens': 385, 'total_tokens': 415, 'completion_tokens': 30}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-405b-instruct'}, id='run-2b937ec3-1167-477b-8509-2d14154a71bc-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current temperature in Edinburgh, Scotland'}, 'id': 'chatcmpl-tool-bc78ce885e32428c9f4848249194279f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 38